In [5]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
% matplotlib inline
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [6]:
## Data Source: https://www.kaggle.com/manjeetsingh/retaildataset
# read the ['Features data set.csv', 'sales data-set.csv', 'stores data-set.csv'] from the /data folder
import os
print("current directory is : " + os.getcwd()) 
print('the /data folder contains:') 
print(os.listdir("./data"))

stores = pd.read_csv('./data/stores data-set.csv')
features = pd.read_csv('./data/Features data set.csv')
sales = pd.read_csv('./data/sales data-set.csv')

## I don't intend to use so instead of replacing na-s with 0 I better drop them all
markdown_cols = features.filter(like='MarkDown').columns
for column in markdown_cols:
     features = features.drop(column,axis=1)

# On features Fill na gaps forward on CPI and Unemployment, Data seems sorted by date already
# https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html
features['CPI'] = features['CPI'].fillna(method='pad')
features['Unemployment'] = features['Unemployment'].fillna(method='pad')

current directory is : C:\Users\User\projects\retail_kaggle
the /data folder contains:
['Features data set.csv', 'sales data-set.csv', 'stores data-set.csv']


In [7]:
# merge left all tables, use only keys from left frame, similar to a SQL left outer join; preserve key order.
    
retail = pd.merge(sales, features, how = 'left', on = ['Store', 'Date', 'IsHoliday'])
retail = pd.merge(retail, stores, how = 'left', on = ['Store'])

# format date in datetime, we need it to extract Year and Year week later to split the data in train, test
retail['Date'] = pd.to_datetime(retail['Date'])
retail['Year'] = pd.DatetimeIndex(retail['Date']).year
retail['Month'] = pd.DatetimeIndex(retail['Date']).month

retail['Year-Week'] = retail['Date'].dt.strftime('%Y-%U')

# I noticed there are Sales figures that are negative, cleanup:
retail = retail[retail['Weekly_Sales']>=0]

# turn IsHoliday into an Integer, useful later for numeric computations
retail['IsHoliday'] = retail['IsHoliday'].astype(int)

# Sort by Date as the data is not ordered by Date
retail = retail.sort_values(by=['Date'])

# checkout the dataset we're going to use further
retail.describe()

,Store,Dept,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Size,Year,Month
count,420285.000000,420285.000000,420285.000000,420285.00000,420285.000000,420285.000000,420285.000000,420285.000000,420285.000000,420285.000000,420285.000000
mean,22.195477,44.242771,16030.329773,0.07034,60.090474,3.360888,171.212152,7.960077,136749.569176,2010.968443,6.478501
std,12.787213,30.507197,22728.500149,0.25572,18.448260,0.458523,39.162280,1.863873,60992.688568,0.796893,3.324688
min,1.000000,1.000000,0.000000,0.00000,-2.060000,2.472000,126.064000,3.879000,34875.000000,2010.000000,1.000000
25%,11.000000,18.000000,2117.560000,0.00000,46.680000,2.933000,132.022667,6.891000,93638.000000,2010.000000,4.000000
50%,22.000000,37.000000,7659.090000,0.00000,62.090000,3.452000,182.350989,7.866000,140167.000000,2011.000000,6.000000
75%,33.000000,74.000000,20268.380000,0.00000,74.280000,3.738000,212.445487,8.567000,202505.000000,2012.000000,9.000000
max,45.000000,99.000000,693099.360000,1.00000,100.140000,4.468000,227.232807,14.313000,219622.000000,2012.000000,12.000000


In [8]:
# Lag data, retail data is already ordered by Date
for i in range(1,9):
    c_name = 'Sales_Lag'+ str(i)
    retail[c_name] = retail.groupby(['Store','Dept'])['Weekly_Sales'].shift(-i)
    
retail.query('Store==1 & Dept==1').head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Type,...,Month,Year-Week,Sales_Lag1,Sales_Lag2,Sales_Lag3,Sales_Lag4,Sales_Lag5,Sales_Lag6,Sales_Lag7,Sales_Lag8
34,1,1,2010-01-10,20094.19,0,71.89,2.603,211.671989,7.838,A,...,1,2010-02,57258.43,16333.14,41595.55,19403.54,16241.78,22517.56,22136.64,26229.21
8,1,1,2010-02-04,57258.43,0,62.27,2.719,210.820450,7.808,A,...,2,2010-05,16333.14,41595.55,19403.54,16241.78,22517.56,22136.64,26229.21,17558.09
21,1,1,2010-02-07,16333.14,0,80.91,2.669,211.223533,7.787,A,...,2,2010-06,41595.55,19403.54,16241.78,22517.56,22136.64,26229.21,17558.09,17596.96
2,1,1,2010-02-19,41595.55,0,39.93,2.514,211.289143,8.106,A,...,2,2010-07,19403.54,16241.78,22517.56,22136.64,26229.21,17558.09,17596.96,16145.35
3,1,1,2010-02-26,19403.54,0,46.63,2.561,211.319643,8.106,A,...,2,2010-08,16241.78,22517.56,22136.64,26229.21,17558.09,17596.96,16145.35,16555.11


In [20]:
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

results_df = []
columns = ['Pred_Dt','Store','Dept','Sales_Lag1','Sales_Lag2','Sales_Lag3','Sales_Lag4','Sales_Lag5','Sales_Lag6','Sales_Lag7','Sales_Lag8']
results_df = pd.DataFrame(columns=columns) 
predict_date = '2012-30'
c = 0
store_list = retail['Store'].unique()
for c_store in store_list:
    dept_list=retail[retail['Store']==c_store]['Dept'].unique()
    for c_dept in dept_list:
        tmp_pred = []
        for i in range(1,9):
            c_name = 'Sales_Lag'+ str(i)
            my_cols = ['Date','Year-Week','Temperature','Fuel_Price','CPI','Unemployment','Size', 'Dept', 'IsHoliday']
            X_cols = ['Temperature','Fuel_Price','CPI','Unemployment','Size', 'Dept', 'IsHoliday']
            my_cols.append(c_name)
            retail_tmp = retail.query('Store=={store} & Dept=={dept}'.format(store=c_store, dept=c_dept))[my_cols]      
            retail_tmp = retail_tmp.dropna()
            # retail_tmp.set_index(retail_tmp['Date'], inplace=True) 
            X = retail_tmp[X_cols]
            y = retail_tmp[c_name]

            c += 1
            clear_output())
            print("Iteration " + str(c) + " Data: " + str(c_store)+ '/' + str(c_dept)  + str(retail_tmp.shape), flush=True)
            
    
            if  X.count()[0]>20:
                forest = RandomForestRegressor(n_estimators=100, criterion='mse', 
                               random_state=42, n_jobs=-1)
                forest.fit(X,y)
                y_pred = forest.predict(X)

                # print(mean_squared_error(y, y_pred))
                # print("R^2: {0:.4f}".format(r2_score(y, y_pred)))
          
                
                # generate predictions for one 
                pred_one = retail_tmp[retail_tmp['Year-Week']==predict_date][X_cols]

 
                if pred_one.count()[0]>0: 
                    y_test_pred = forest.predict(pred_one)
                   # print(y_test_pred)
               
                else:
                        y_test_pred =0
            else:
                y_test_pred =0
            
            tmp_pred.append(float(y_test_pred))    
                
        results_df = results_df.append({"Pred_Dt":predict_date,"Store": c_store,'Dept':c_dept,
                                        'Sales_Lag1':tmp_pred[0],
                                        'Sales_Lag2':tmp_pred[1],
                                        'Sales_Lag3':tmp_pred[2],
                                        'Sales_Lag4':tmp_pred[3],
                                        'Sales_Lag5':tmp_pred[4],
                                        'Sales_Lag6':tmp_pred[5],
                                        'Sales_Lag7':tmp_pred[6],
                                        'Sales_Lag8':tmp_pred[7]}
                                       , ignore_index=True)
        # print('On' + str(predict_date) + 'the predicted ' + str(c_name) + ' for Store: '+ str(c_store) + " dept " + str(c_dept) + " is:" + str(float(y_test_pred)))



SyntaxError: invalid syntax (<ipython-input-20-063944cf8b9a>, line 26)

In [10]:
Sales_col = results_df.columns[results_df.columns.str.contains(pat = 'Lag')]
results_df.set_index('Store')
tmp_res= results_df[Sales_col].groupby(results_df.index).sum(axis=1).sum(axis=1)

In [11]:
print('Prediction for each store for the next 8 weeks, as of year-week: ' + str(predict_date))
print(tmp_res.astype(int))

Prediction for each store for the next 8 weeks, as of year-week: 2012-30
0       258369
1        57182
2       614907
3       176476
4       300346
5         5078
6       118295
7        35908
8       405089
9       767022
10      326601
11      284289
12      238144
13       43660
14        3657
15       64338
16      257534
17       22596
18      175401
19       76462
20      716113
21       56792
22      101301
23       37232
24       37315
25         459
26       24756
27       25349
28      495109
29     1219819
        ...   
447      11924
448          0
449          0
450      49078
451      41225
452      99052
453     242990
454     413169
455       9964
456      34264
457      33703
458      10406
459      65827
460     100919
461      15287
462       6676
463     191947
464      48623
465      72475
466     237002
467      20009
468      25520
469      31275
470       4989
471      42123
472      76842
473      59670
474      78242
475      58673
476      28010
Length: 477,

In [12]:
print('Prediction for the entire network for the next 8 weeks, as of year-week: ' + str(predict_date))
print(tmp_res.astype(int).sum(axis=0))

Prediction for the entire network for the next 8 weeks, as of year-week: 2012-30
64857645
